# 4-D quantum spacetime with spin network states
$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}
$

## Problem description
Each quantum tetrahedra can be written in the form 
$$
\ket{T} = \cos(\frac{\theta}{2}) \ket{0_L} + e^{i\phi}\sin(\frac{\theta}{2}) \ket{1_L}
$$
$$ 
A = \braket{W}{T}
$$

## Setup
Load code from Cirq and our LQG directory

In [474]:
import cirq
import recirq
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import fnmatch
os.chdir('LQG')
import tetrahedra_num as tn
import tetrahedra_circuits as tc
import postprocess as post
import imp
imp.reload(tn)
imp.reload(tc)
imp.reload(post)
os.chdir('..')

## Hardware layout
This project was executed on the Google Sycamore processor (Weber specifically), whose 2D-qubit layout is pictured below.

In [475]:
print(cirq.google.devices.Sycamore)

                                             (0, 5)───(0, 6)
                                             │        │
                                             │        │
                                    (1, 4)───(1, 5)───(1, 6)───(1, 7)
                                    │        │        │        │
                                    │        │        │        │
                           (2, 3)───(2, 4)───(2, 5)───(2, 6)───(2, 7)───(2, 8)
                           │        │        │        │        │        │
                           │        │        │        │        │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)───(3, 6)───(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │        │        │        │        │
                  │        │        │        │        │        │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)───(4, 6)───(4, 7)───(4, 8)───(4, 9)
         │        │        │        │        │        │   

# Experiment 1
The first submission contained ~3.5 million circuit samples.

First, we experimentally determined the fidelity of preparing the states $\ket{0_L}$ and $\ket{1_L}$.

Second, we experimentally calculated the probability of ending in the $\bf{\ket{0}}$ state for a monopole network, dipole network, and pentagram network, which is also called a 4-simplice and is the boundary state for a 4-d spacetime.

Lastly, we glued two 4-simplices together at two tetrahedra and calculated the probability of ending in $\bf{\ket{0}}$. This experiment is more speculative and has not been performed before.

## Load data

In [476]:
results, sample_names = post.load(tc.project_name + '/lqg_ha_cr_measurements_submission_1.json')
sim_results, sample_names = post.load(tc.project_name + '/lqg_ha_cr_test_measurements.json')
err_results, _ = post.load(tc.project_name + '/lqg_ha_cr_error.json')

## State preparation fidelity

In [477]:
# load the samples as wavefunctions
zero_prep_sample = post.SampleWavefunction(results['zero_fidelity'],4)
one_prep_sample = post.SampleWavefunction(results['one_fidelity'],4)
# get the error
zero_prep_fidelity = zero_prep_sample.error(tn.L_zero)
one_prep_fidelity = one_prep_sample.error(tn.L_one)

# post-selection
zero_prep_sample.post_select(post.keepfn_L_zero)
one_prep_sample.post_select(post.keepfn_even)
# get the error
zero_prep_post = zero_prep_sample.error(tn.L_zero)
one_prep_post = one_prep_sample.error(tn.L_one)

State preparation was tested with 1,000 samples, so larger distance from the wavefunction is expected.

Without any sort of error management, we get the following fidelities:

$\ket{0_L}$: {{ str(round(zero_prep_fidelity,4))}}

$\ket{1_L}$: {{ str(round(one_prep_fidelity,4)) }}

With post-selection, we get the following fidelities:

$\ket{0_L}$: {{str(round(zero_prep_post,4))}}

$\ket{1_L}$: {{str(round(one_prep_post,4)) }}

## Error tracking - where did my amplitudes go?

In [478]:
exphist = results['zero_two_transition_04152637_0']
simhist = sim_results['zero_two_transition_04152637_0']
#errhist = err_results['zero_two_transition_0']

simsamples = post.SampleWavefunction(simhist,8)
st = simsamples.state_subset([0],[None],verbose=False)
d = {}
for key,value in st.items():
    try:
        d[key.count('1')] += value
    except:
        d[key.count('1')] = value
print(2**-8)
print(simsamples.amplitude)

#expwvfn = post.SampleWavefunction(exphist,8)
#for i in range(8):
#    s = expwvfn.state_subset([i],[i+1],verbose=False)
#    if len(s) == 1:
#        print(s)
#expwvfn.post_select(post.keepfn_even)
#expwvfn.state_subset([2,6],[4,8])
#simwvfn = post.SampleWavefunction(simhist,8)
#simwvfn.state_subset([2,6],[4,8])
#errwvfn = post.SampleWavefunction(errhist,8)
#errwvfn.state_subset([2,6],[4,8])


#print('<sim|exp>',simwvfn.error(expwvfn.wavefn))
#print('<sim|err>',simwvfn.error(errwvfn.wavefn))
#print('<exp|err>',expwvfn.error(errwvfn.wavefn))
#print('sim',simwvfn.amplitude)
#print('exp',expwvfn.amplitude)
#print('err',errwvfn.amplitude)
#print('<|err>',simwvfn.error(errwvfn.wavefn)

subsets of [0] to [None]
0.00390625
0.0606


## Monopole spin network

In [495]:
# get all samples of the name 'zero_transition_*' and 'one_transition_*'
zmono = post.Experiment(results, 'zero_transition_*', 8)
a = zmono.amplitude()[1]
zmono.post_select(post.keepfn_even)
b = zmono.amplitude()[1]
print(a,b)

omono = post.Experiment(results, 'one_transition_*', 8)
#print((a+b)/2)
print(omono.amplitude())

0.2354 0.2941749351203427
([0.345, 0.335, 0.341, 0.343, 0.351], 0.34299999999999997)


## Dipole spin network

In [480]:
# get all samples of 04152637
zdipole = post.Experiment(results,'zero_two_transition_04152637*', 8)
odipole = post.Experiment(results,'one_two_transition_04152637*', 8)
print(zdipole.amplitude())
print(odipole.amplitude())

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.0)
([0], 0.0)


In [493]:
# get all samples of 04172536
zdialtdipole = post.Experiment(results, 'zero_two_transition_04172536*', 8)#, qubitmap=zdialtmap)
a = zdialtdipole.amplitude()[1]
print(zdialtdipole.probability(2**8-1))
zdialtdipole.post_select(post.keepfn_even)
b = zdialtdipole.amplitude()[1]
avg = (a+b)/2
print(avg)
print(zdialtdipole.probability(2**8-1))
print((1/64-avg)*64)

([0.0068, 0.0064, 0.0069, 0.0064, 0.0072, 0.0075, 0.0085, 0.0078, 0.0059, 0.007], 0.00704)
0.015099187616127763
([0.011037169290699561, 0.010420058612829698, 0.011166855478232724, 0.010291043576137643, 0.011883149034494141, 0.012212994626282364, 0.0136327185244587, 0.012761780104712041, 0.009577922077922078, 0.011347057869995137], 0.01143307491957641)
0.03365199256782314


In [482]:
# get all samples of 07152634
odialtmap = [(0,6),(3,5),(6,3),(5,0)]
odialt_samples = post.Sample(results, 'onetwo_transition_04172536*', 8)#, qubitmap=odialtmap)
print(odialt_samples.amplitude)
odialt_samples.post_select(post.keepfn_even)

TypeError: unsupported operand type(s) for +=: 'int' and 'dict'

## 4-simplex

In [ ]:


print(10_000/1_500_500)
osimp_samples = post.glob_samples(results, 'zero_five_transition*', 20)
osimp_samples.post_select(post.keepfn_even)
print(osimp_samples.amplitude)
print(1/2**9)

s = 0
for key,value in osimp_samples.hist.items():
    state = post.split_state(key,20)
    if '0000' in state:
        s += value
print(s/osimp_samples.samples)
print(osimp_samples.samples)

In [ ]:
oglue_samples = post.glob_samples(results, 'zero_ten_transition*', 40)
print(oglue_samples.probability(2**40-1))
for key,value in oglue_samples.hist.items():
    if value > 1:
        print(tc.binformat(40).format(key))
print('done')

In [ ]:
q = cirq.LineQubit.range(4)
c = cirq.Circuit(cirq.X.on_each(q),
                 cirq.H.on_each(q[0],q[2]),
                 cirq.CNOT(q[0],q[1]),
                 cirq.CNOT(q[2],q[3]),
                 cirq.CNOT(q[0],q[2]),
                 cirq.reset(q[3]),
                 cirq.H.on_each(q[:2]),
                 cirq.X.on_each(q)
                )

print(c[0].qubits)
#print(cirq.Simulator().simulate(c))
#c.append(cirq.measure(*q,key='z'))
#hist = tc.final_state_simulation(c,100000).histogram(key='z')
#for key,value in hist.items():
#    print(tc.binformat(4).format(key))
SVGCircuit(c)